<a href="https://colab.research.google.com/github/lazy-wav/Vastra-AI/blob/main/copy_of_vastra_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!pip install keras
!pip install tqdm
!pip install pickle
!pip install matplotlib-venn
!pip install kaggle

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d ronakbokaria/myntra-products-dataset

Dataset URL: https://www.kaggle.com/datasets/ronakbokaria/myntra-products-dataset
License(s): CC0-1.0
 99% 114M/115M [00:01<00:00, 74.3MB/s]
100% 115M/115M [00:01<00:00, 67.2MB/s]


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import keras.layers
import keras.applications
import tensorflow.keras.applications.resnet50

In [ ]:
import tensorflow
import keras
from numpy.linalg import norm
from tqdm import tqdm
import pickle

In [ ]:
# extracting the data zipped file
from zipfile import ZipFile
dataset='/content/myntra-products-dataset.zip'
with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print ('Dataset extracted successfully!')

Dataset extracted successfully!


In [ ]:
from keras.preprocessing import image
from keras.layers import GlobalMaxPooling2D
from keras.applications.resnet50 import ResNet50,preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
articles = pd.read_csv("/content/myntra202305041052.csv")
customers = pd.read_csv("/content/myntra202305041052.csv")
transactions = pd.read_csv("/content/myntra202305041052.csv")

In [ ]:
#loading data from csv_file to pd
data=pd.read_csv('/content/myntra202305041052.csv', encoding ='ISO-8859-1')

In [ ]:
data.head()

,id,name,img,asin,price,mrp,rating,ratingTotal,discount,seller,purl
0,1,Men Solid Oversized Cotton,"https://assets.myntassets.com/f_webp,dpr_1.0,q...",-,532.0,1299.0,4.1,5300,59,Difference of Opinion,https://www.myntra.com/tshirts/difference-of-o...
1,2,Men Cotton Pure Cotton T-shirt,"https://assets.myntassets.com/f_webp,dpr_1.0,q...",-,274.0,499.0,4.2,25400,45,Roadster,https://www.myntra.com/tshirts/roadster/roadst...
2,3,Women Pure Cotton T-shirt,"https://assets.myntassets.com/f_webp,dpr_1.0,q...",-,551.0,1199.0,4.5,3400,54,DILLINGER,https://www.myntra.com/tshirts/dillinger/dilli...
3,4,Typography Print T-shirt,"https://assets.myntassets.com/f_webp,dpr_1.0,q...",-,296.0,1099.0,4.0,18400,73,Huetrap,https://www.myntra.com/tshirts/huetrap/huetrap...
4,5,Printed Round Neck Pure Cotton T-shirt,"https://assets.myntassets.com/f_webp,dpr_1.0,q...",-,494.0,899.0,4.2,3800,45,Roadster,https://www.myntra.com/tshirts/roadster/roadst...


In [ ]:


# Example logic to categorize items (you may need to adjust this based on your actual data)
def categorize_item(name):
    if 'top' in name.lower() or 't-shirt' in name.lower():
        return 'tops'
    elif 'bottom' in name.lower() or 'pant' in name.lower():
        return 'bottoms'
    elif 'dress' in name.lower():
        return 'dresses'
    elif 'jewel' in name.lower() or 'earring' in name.lower() or 'necklace' in name.lower():
        return 'jewelry'
    else:
        return 'other'

data['category'] = data['name'].apply(categorize_item)


In [ ]:
import os
import requests
from PIL import Image
from io import BytesIO

image_dir = '/content/images'

# Create directories for each category
categories = data['category'].unique()
for category in categories:
    os.makedirs(os.path.join(image_dir, category), exist_ok=True)

# Function to download an image from a URL
def download_image(url, save_path):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    img.save(save_path)

# Download all images into respective category folders
for index, row in data.iterrows():
    img_url = row['img']
    img_id = row['id']
    category = row['category']
    img_path = os.path.join(image_dir, category, f'{img_id}.jpg')
    try:
        download_image(img_url, img_path)
    except Exception as e:
        print(f"Failed to download image {img_id}: {e}")


Failed to download image 71: cannot identify image file <_io.BytesIO object at 0x79d9e875fb50>
Failed to download image 91: cannot identify image file <_io.BytesIO object at 0x79d9a2ea6a70>
Failed to download image 757: cannot identify image file <_io.BytesIO object at 0x79d9e858c130>
Failed to download image 764: cannot identify image file <_io.BytesIO object at 0x79d9a6007240>
Failed to download image 1128: cannot identify image file <_io.BytesIO object at 0x79d9e86d7dd0>


KeyboardInterrupt: 

In [ ]:

import pandas as pd
import os

image_dir = '/content/images'
categories = ['bottoms', 'dresses', 'jewelry', 'other', 'tops']
data = []
for category in categories:
    category_dir = os.path.join(image_dir, category)
    if os.path.exists(category_dir) and os.path.isdir(category_dir):
        for img_file in os.listdir(category_dir):
            if img_file.endswith(('.jpg', '.jpeg', '.png')):
                data.append((os.path.join(category_dir, img_file), category))
    else:
        print(f"Directory {category_dir} does not exist. Skipping.")

df = pd.DataFrame(data, columns=['filepath', 'category'])


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    x_col='filepath',
    y_col='category',
    target_size=(224, 224),
    batch_size=32,
    class_mode='input',  # Use 'input' because we only care about images
    subset='training'
)

validation_generator = datagen.flow_from_dataframe(
    dataframe=df,
    x_col='filepath',
    y_col='category',
    target_size=(224, 224),
    batch_size=32,
    class_mode='input',
    subset='validation'
)


Found 904 validated image filenames.
Found 226 validated image filenames.


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

class SiameseDataGenerator(Sequence):
    def __init__(self, df, batch_size, input_shape, shuffle=True):
        self.df = df
        self.batch_size = batch_size
        self.input_shape = input_shape
        self.shuffle = shuffle
        self.label_encoder = LabelEncoder()
        self.df['encoded_category'] = self.label_encoder.fit_transform(self.df['category'])
        self.indices = np.arange(len(self.df))
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.df) / self.batch_size))

    def __getitem__(self, index):
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        pairs, labels = self.__data_generation(indices)
        return pairs, labels

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

    def __data_generation(self, indices):
        pairs = [np.zeros((self.batch_size, *self.input_shape)) for _ in range(2)]
        labels = np.zeros((self.batch_size,))

        for i, index in enumerate(indices):
            img1_path, category1 = self.df.iloc[index][['filepath', 'encoded_category']]
            img2_path, category2 = self.__get_random_image(category1)

            pairs[0][i] = self.__load_image(img1_path)
            pairs[1][i] = self.__load_image(img2_path)
            labels[i] = 1 if category1 == category2 else 0

        return pairs, labels

    def __get_random_image(self, category):
        same_category_df = self.df[self.df['encoded_category'] == category]
        diff_category_df = self.df[self.df['encoded_category'] != category]

        if np.random.rand() > 0.5:
            return same_category_df.sample(1)[['filepath', 'encoded_category']].values[0]
        else:
            return diff_category_df.sample(1)[['filepath', 'encoded_category']].values[0]

    def __load_image(self, img_path):
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=self.input_shape[:2])
        img = tf.keras.preprocessing.image.img_to_array(img)
        img /= 255.0
        return img


In [ ]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

train_generator = SiameseDataGenerator(train_df, batch_size=32, input_shape=(224, 224, 3))
validation_generator = SiameseDataGenerator(val_df, batch_size=32, input_shape=(224, 224, 3))


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
import tensorflow as tf

# Load pre-trained ResNet50 model + higher level layers
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)  # Add a trainable dense layer
feature_model = Model(inputs=base_model.input, outputs=x)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Define the Siamese network architecture
input_shape = (224, 224, 3)
input_a = Input(shape=input_shape, name='input_a')
input_b = Input(shape=input_shape, name='input_b')

# Create embeddings for both inputs
embedding_a = feature_model(input_a)
embedding_b = feature_model(input_b)

# Define the distance metric
distance = tf.keras.layers.Lambda(lambda embeddings: tf.norm(embeddings[0] - embeddings[1], axis=1))([embedding_a, embedding_b])

# Create the Siamese network
siamese_net = Model(inputs=[input_a, input_b], outputs=distance)

# Compile the model
siamese_net.compile(optimizer='adam', loss='binary_crossentropy')


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import Sequence
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

class SiameseDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, df, batch_size, input_shape, shuffle=True):
        self.df = df
        self.batch_size = batch_size
        self.input_shape = input_shape
        self.shuffle = shuffle
        self.label_encoder = LabelEncoder()
        self.df['encoded_category'] = self.label_encoder.fit_transform(self.df['category'])
        self.indices = np.arange(len(self.df))
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.df) / self.batch_size))

    def __getitem__(self, index):
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        pairs, labels = self.__data_generation(indices)
        return pairs, labels

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

    def __data_generation(self, indices):
        pairs = [np.zeros((self.batch_size, *self.input_shape)) for _ in range(2)]
        labels = np.zeros((self.batch_size,))

        for i, index in enumerate(indices):
            img1_path, category1 = self.df.iloc[index][['filepath', 'encoded_category']]
            img2_path, category2 = self.__get_random_image(category1)

            pairs[0][i] = self.__load_image(img1_path)
            pairs[1][i] = self.__load_image(img2_path)
            labels[i] = 1 if category1 == category2 else 0

        return {'input_a': pairs[0], 'input_b': pairs[1]}, labels

    def __get_random_image(self, category):
        same_category_df = self.df[self.df['encoded_category'] == category]
        diff_category_df = self.df[self.df['encoded_category'] != category]

        if np.random.rand() > 0.5:
            return same_category_df.sample(1)[['filepath', 'encoded_category']].values[0]
        else:
            return diff_category_df.sample(1)[['filepath', 'encoded_category']].values[0]

    def __load_image(self, img_path):
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=self.input_shape[:2])
        img = tf.keras.preprocessing.image.img_to_array(img)
        img /= 255.0
        return img

# Ensure TensorFlow compatible output signature
def generator_output_signature(input_shape):
    return ({'input_a': tf.TensorSpec(shape=(None, *input_shape), dtype=tf.float32),
             'input_b': tf.TensorSpec(shape=(None, *input_shape), dtype=tf.float32)},
            tf.TensorSpec(shape=(None,), dtype=tf.float32))

# Create a function to wrap the data generator for use in tf.data.Dataset
def create_dataset(df, batch_size, input_shape):
    return tf.data.Dataset.from_generator(
        lambda: SiameseDataGenerator(df, batch_size, input_shape),
        output_signature=generator_output_signature(input_shape)
    )

# Usage example
input_shape = (224, 224, 3)
batch_size = 32

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
train_dataset = create_dataset(train_df, batch_size, input_shape)
val_dataset = create_dataset(val_df, batch_size, input_shape)

# Now the datasets can be used with the Siamese network
siamese_net.fit(train_dataset, epochs=10, validation_data=val_dataset)


Epoch 1/10
   1224/Unknown 16406s 13s/step - loss: 0.2797

In [ ]:
from google.colab import drive
drive.mount('/content/drive')